### Imports 

In [44]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler, StandardScaler


import matplotlib.pyplot as plt
%matplotlib inline

### Read data for all business

In [45]:
businesses = pd.read_csv('../data/yelp_business.csv')
businesses.head(5)

,business_id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories
0,FYWN1wneV18bWNgQjJ2GNg,"""Dental by Design""",NaN,"""4855 E Warner Rd, Ste B9""",Ahwatukee,AZ,85044,33.330690,-111.978599,4.0,22,1,Dentists;General Dentistry;Health & Medical;Or...
1,He-G7vWjzVUysIKrfNbPUQ,"""Stephen Szabo Salon""",NaN,"""3101 Washington Rd""",McMurray,PA,15317,40.291685,-80.104900,3.0,11,1,Hair Stylists;Hair Salons;Men's Hair Salons;Bl...
2,KQPW8lFf1y5BT2MxiSZ3QA,"""Western Motor Vehicle""",NaN,"""6025 N 27th Ave, Ste 1""",Phoenix,AZ,85017,33.524903,-112.115310,1.5,18,1,Departments of Motor Vehicles;Public Services ...
3,8DShNS-LuFqpEWIp0HxijA,"""Sports Authority""",NaN,"""5000 Arizona Mills Cr, Ste 435""",Tempe,AZ,85282,33.383147,-111.964725,3.0,9,0,Sporting Goods;Shopping
4,PfOCPjBrlQAnz__NXj9h_w,"""Brick House Tavern + Tap""",NaN,"""581 Howe Ave""",Cuyahoga Falls,OH,44221,41.119535,-81.475690,3.5,116,1,American (New);Nightlife;Bars;Sandwiches;Ameri...


### Filter restaurants

In [46]:
restaurants = businesses[businesses['categories'].str.contains("Restaurants")]
restaurants.head(5)

,business_id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories
4,PfOCPjBrlQAnz__NXj9h_w,"""Brick House Tavern + Tap""",NaN,"""581 Howe Ave""",Cuyahoga Falls,OH,44221,41.119535,-81.475690,3.5,116,1,American (New);Nightlife;Bars;Sandwiches;Ameri...
5,o9eMRCWt5PkpLDE0gOPtcQ,"""Messina""",NaN,"""Richterstr. 11""",Stuttgart,BW,70567,48.727200,9.147950,4.0,5,1,Italian;Restaurants
10,XOSRcvtaKc_Q5H1SAzN20A,"""East Coast Coffee""",NaN,"""737 West Pike St""",Houston,PA,15342,40.241548,-80.212815,4.5,3,0,Breakfast & Brunch;Gluten-Free;Coffee & Tea;Fo...
14,fNMVV_ZX7CJSDWQGdOM8Nw,"""Showmars Government Center""",Uptown,"""600 E 4th St""",Charlotte,NC,28202,35.221647,-80.839345,3.5,7,1,Restaurants;American (Traditional)
15,l09JfMeQ6ynYs5MCJtrcmQ,"""Alize Catering""",Yonge and Eglinton,"""2459 Yonge St""",Toronto,ON,M4P 2H6,43.711399,-79.399339,3.0,12,0,Italian;French;Restaurants


### Read other attributes of all businesses (deprecated as of now)

In [47]:
# business_attr = pd.read_csv('data/yelp_business_attributes.csv')

### Merge filtered restaurants with above attributes

In [48]:
# restaurant_attr = restaurants.merge(business_attr, left_on='business_id', right_on='business_id', how = 'inner')
restaurant_attr = restaurants

### Look at the distribution of open and closed restaurants 

In [49]:
restaurant_attr['is_open'].value_counts()

1    40394
0    14224
Name: is_open, dtype: int64

### Look at Top 5 cities with most open restaurants

In [50]:
restaurant_attr[(restaurant_attr['is_open'] == 1)].\
    groupby('city')['business_id'].count().reset_index().\
        sort_values('business_id', ascending = False)[:5]



,city,business_id
678,Toronto,4968
302,Las Vegas,3990
400,Montréal,2596
495,Phoenix,2571
98,Charlotte,1860


### We will focus on 1 city for now. Filter open restaurants  in Toronto 

In [51]:
city = 'Toronto'
restaurants_in_city = restaurant_attr[(restaurant_attr['is_open'] == 1) & (restaurant_attr['city'] == city)]

restaurants_in_city.to_csv('../data/restaurants_in_'+city+'.csv', index = False)

### Use this file and checkin data in 'Parse Checkin Data'
look at Parse Checkin Data.ipynb

### Read checkin data and create weekend flag

In [52]:
business_checkin = pd.read_csv('../data/restaurant_Toronto_checkin.csv')
# restaurant_checkin = pd.merge(restaurant_attr['business_id'], business_checkin, 
#          left_on = 'business_id', right_on = 'business_id', how = 'inner')
business_checkin['is_weekend'] = business_checkin['day_of_week'].\
                            apply(lambda x : 1 if x == 'Saturday' or x == 'Sunday' or x == 'Friday' else 0)

### Function to create categorical variable - Morning, Afternoon, Evening, Night from time 

In [53]:
def create_time_of_day(x):
    hour = int(x)
    if 6<= hour <= 11:
        return 'Morning'
    elif 12<= hour <= 15:
        return 'Afternoon'
    elif 16<= hour <= 20:
        return 'Evening'
    elif 20<=hour <= 24 or 0<=hour<=2:
        return 'Night'
    else:
        return 'Late Night'
business_checkin['time_of_day'] = business_checkin['hour'].apply(lambda x: create_time_of_day(x))

### Feature Engineering to capture different checkin patterns 

In [54]:
#Average checkins on weekdays and weekends

feature_1 = pd.pivot_table(business_checkin, values='checkins', index=['business_id'],
                     columns=['is_weekend'], aggfunc=np.mean).reset_index()
feature_1.columns = ['business_id', 'avg_checkins_weekday', 'avg_checkins_weekend']
# feature_1

In [55]:
#Average checkins on different day of the week

feature_2 = pd.pivot_table(business_checkin, values='checkins', index=['business_id'],
                     columns=['day_of_week'], aggfunc=np.mean).reset_index()
feature_2.columns = ['business_id', 'avg_checkins_Fri', 'avg_checkins_Mon', 'avg_checkins_Sat', 'avg_checkins_Sun',
                    'avg_checkins_Thu', 'avg_checkins_Tue', 'avg_checkins_Wed']
# feature_2

In [56]:
#Average checkins on different time of the day

feature_3 = pd.pivot_table(business_checkin, values='checkins', index=['business_id'],
                     columns=['time_of_day'], aggfunc=np.mean).reset_index()
feature_3.columns = ['business_id', 'Avg_checkins_Afternoon', 'Avg_checkins_Evening', 'Avg_checkins_Late_Night', 
                     'Avg_checkins_Morning', 'Avg_checkins_Night']
# feature_3

In [57]:
#Average checkins on weekdays vs. weekends broken by different times of the day

feature_4 = pd.pivot_table(business_checkin, values='checkins', index=['business_id'],
                     columns=['is_weekend','time_of_day'], aggfunc=np.mean).reset_index()


feature_4.columns = [str(col[0])+ '_'+col[1] for col in feature_4.columns]
feature_4.rename(columns={'business_id_':'business_id'}, inplace=True)

# feature_4

### Merge all features and attributes

In [58]:
feature_1_2 = pd.merge(feature_1, feature_2, 
         left_on = 'business_id', right_on = 'business_id', how = 'inner')
feature_1_2_3 = pd.merge(feature_1_2, feature_3, 
         left_on = 'business_id', right_on = 'business_id', how = 'inner')

feature_1_2_3_4 = pd.merge(feature_1_2_3, feature_4, 
         left_on = 'business_id', right_on = 'business_id', how = 'inner')
restaurant_attr_all = pd.merge(restaurant_attr, feature_1_2_3_4, 
         left_on = 'business_id', right_on = 'business_id', how = 'inner')

In [59]:
pd.set_option('display.max_columns', 200)
restaurant_attr_all.head(5)

,business_id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories,avg_checkins_weekday,avg_checkins_weekend,avg_checkins_Fri,avg_checkins_Mon,avg_checkins_Sat,avg_checkins_Sun,avg_checkins_Thu,avg_checkins_Tue,avg_checkins_Wed,Avg_checkins_Afternoon,Avg_checkins_Evening,Avg_checkins_Late_Night,Avg_checkins_Morning,Avg_checkins_Night,0_Afternoon,0_Evening,0_Late Night,0_Morning,0_Night,1_Afternoon,1_Evening,1_Late Night,1_Morning,1_Night
0,1K4qrnfyzKzGgJPBEcJaNQ,"""Chula Taberna Mexicana""",Leslieville,"""1058 Gerrard Street E""",Toronto,ON,M4M 3A6,43.669256,-79.335902,3.5,39,1,Tiki Bars;Nightlife;Mexican;Restaurants;Bars,1.222222,1.689655,1.125,1.25,1.900000,1.909091,1.25,1.285714,1.000000,1.75,1.266667,1.25,NaN,1.666667,1.0,1.250000,1.0,NaN,1.272727,2.0,1.272727,1.5,NaN,2.000000
1,nbhBRhZtdaZmMMeb2i02pg,"""Sunnyside Grill""",NaN,"""2777 Steeles Avenue W""",Toronto,ON,M3J 3K5,43.781816,-79.490433,5.0,3,1,Restaurants;Breakfast & Brunch,1.000000,1.166667,1.000,1.00,1.000000,1.500000,NaN,1.000000,1.000000,1.00,1.200000,NaN,1.0,NaN,1.0,1.000000,NaN,1.0,NaN,1.0,1.333333,NaN,NaN,NaN
2,FXHfcFVEfI1vVngW2gVOpw,"""Bampot House of Tea & Board Games""",NaN,"""201 Harbord Street""",Toronto,ON,M5S 1H6,43.661582,-79.408878,4.0,55,1,Coffee & Tea;Restaurants;Food;Mediterranean;Te...,1.352941,1.625000,1.750,1.00,1.333333,1.818182,1.75,1.333333,1.333333,1.00,1.545455,1.75,1.0,1.500000,1.0,1.666667,1.0,NaN,1.333333,NaN,1.500000,2.0,1.0,1.666667
3,VXH7zXcZzXlmAVN8GSjGRQ,"""Thai Express""",NaN,"""4700 Keele Street""",Toronto,ON,M3J 1P3,43.774878,-79.494619,3.0,5,1,Imported Food;Specialty Food;Thai;Ethnic Food;...,1.000000,1.000000,NaN,NaN,1.000000,NaN,NaN,1.000000,NaN,NaN,1.000000,NaN,NaN,1.000000,NaN,1.000000,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,1.000000
4,dTWfATVrBfKj7Vdn0qWVWg,"""Flavor Cuisine""",Scarborough,"""8 Glen Watford Drive""",Toronto,ON,M1S 2C1,43.787061,-79.276166,3.0,6,1,Restaurants;Chinese;Food Court,1.000000,1.000000,1.000,1.00,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,1.000000,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000


### Look at distribution of count of reviews 

In [60]:
percentile_value = np.percentile(restaurant_attr_all['review_count'], 70)

print('Count of restaurants having reviews more than', percentile_value,':', len(restaurant_attr_all[restaurant_attr_all['review_count'] > percentile_value]))

Count of restaurants having reviews more than 50.0 : 1425


### Identify Top 30 popular cuisines

In [61]:
from collections import Counter

all_categories = restaurant_attr_all['categories'].apply(lambda x: x.split(';')).tolist()
all_categories_flat_list = [item for sublist in all_categories for item in sublist]

category_counts = Counter(all_categories_flat_list)
print(category_counts.most_common(30))


[('Restaurants', 4794), ('Food', 1082), ('Nightlife', 681), ('Bars', 665), ('Sandwiches', 461), ('Breakfast & Brunch', 424), ('Canadian (New)', 412), ('Cafes', 399), ('Chinese', 386), ('Coffee & Tea', 374), ('Pizza', 367), ('Italian', 351), ('Fast Food', 339), ('Japanese', 325), ('Burgers', 282), ('American (Traditional)', 239), ('Sushi Bars', 236), ('Middle Eastern', 185), ('Indian', 179), ('Mediterranean', 177), ('Pubs', 176), ('Mexican', 175), ('Asian Fusion', 175), ('Thai', 171), ('Korean', 167), ('Seafood', 159), ('Bakeries', 158), ('Specialty Food', 156), ('Event Planning & Services', 156), ('Salad', 151)]


### Create flag for different cuisines 

In [62]:

def create_restaurant_cuisine_flag(df, cuisine):
    df_cuisine = df[df['categories'].str.contains(cuisine)][['business_id', 'cuisine_flag']]
    df_cuisine.columns = ['business_id', cuisine]
    return df_cuisine
    
restaurant_attr_all['cuisine_flag'] = 1
restaurants_in_Toronto_w_Nightlife = create_restaurant_cuisine_flag(restaurant_attr_all, 'Nightlife')
restaurants_in_Toronto_w_Bar = create_restaurant_cuisine_flag(restaurant_attr_all, 'Bars')
restaurants_in_Toronto_w_Canadian = create_restaurant_cuisine_flag(restaurant_attr_all, 'Canadian')
restaurants_in_Toronto_w_Chinese = create_restaurant_cuisine_flag(restaurant_attr_all, 'Chinese')
restaurants_in_Toronto_w_Italian = create_restaurant_cuisine_flag(restaurant_attr_all, 'Italian')
restaurants_in_Toronto_w_Japanese = create_restaurant_cuisine_flag(restaurant_attr_all, 'Japanese')
restaurants_in_Toronto_w_American = create_restaurant_cuisine_flag(restaurant_attr_all, 'American')
restaurants_in_Toronto_w_Indian = create_restaurant_cuisine_flag(restaurant_attr_all, 'Indian')
restaurants_in_Toronto_w_Mexican = create_restaurant_cuisine_flag(restaurant_attr_all, 'Mexican')
restaurants_in_Toronto_w_Thai = create_restaurant_cuisine_flag(restaurant_attr_all, 'Thai')


### Merging cuisine flags 

In [63]:
df_1 = pd.merge(restaurant_attr_all, restaurants_in_Toronto_w_Nightlife, 
         left_on = 'business_id', right_on = 'business_id', how = 'left')
df_2 = pd.merge(df_1, restaurants_in_Toronto_w_Bar, 
         left_on = 'business_id', right_on = 'business_id', how = 'left')
df_3 = pd.merge(df_2, restaurants_in_Toronto_w_Canadian, 
         left_on = 'business_id', right_on = 'business_id', how = 'left')
df_4 = pd.merge(df_3, restaurants_in_Toronto_w_Chinese, 
         left_on = 'business_id', right_on = 'business_id', how = 'left')
df_5 = pd.merge(df_4, restaurants_in_Toronto_w_Italian, 
         left_on = 'business_id', right_on = 'business_id', how = 'left')
df_6 = pd.merge(df_5, restaurants_in_Toronto_w_Japanese, 
         left_on = 'business_id', right_on = 'business_id', how = 'left')
df_7 = pd.merge(df_6, restaurants_in_Toronto_w_American, 
         left_on = 'business_id', right_on = 'business_id', how = 'left')
df_8 = pd.merge(df_7, restaurants_in_Toronto_w_Indian, 
         left_on = 'business_id', right_on = 'business_id', how = 'left')
df_9 = pd.merge(df_8, restaurants_in_Toronto_w_Mexican, 
         left_on = 'business_id', right_on = 'business_id', how = 'left')
df_10 = pd.merge(df_9, restaurants_in_Toronto_w_Thai, 
         left_on = 'business_id', right_on = 'business_id', how = 'left')

### Final columms and filters

In [64]:
relevant_cols = ['business_id','name','stars','review_count','avg_checkins_weekday', 'avg_checkins_weekend',
                'avg_checkins_Fri', 'avg_checkins_Mon', 'avg_checkins_Sat', 'avg_checkins_Sun',
                'avg_checkins_Thu', 'avg_checkins_Tue', 'avg_checkins_Wed', 'Avg_checkins_Afternoon',
                'Avg_checkins_Evening', 'Avg_checkins_Late_Night', 'Avg_checkins_Morning', 'Avg_checkins_Night',
                '0_Afternoon', '0_Evening', '0_Late Night', '0_Morning', '0_Night', '1_Afternoon', 
                '1_Evening', '1_Late Night', '1_Morning', '1_Night',
                'Nightlife', 'Bars', 'Canadian', 'Chinese', 'Italian', 'Japanese','American', 'Indian', 
                'Mexican', 'Thai']

X = df_10.loc[:,relevant_cols]
X.fillna(0, inplace = True)

relevant_cuisines = ['Nightlife', 'Bars', 'Canadian', 'Chinese', 'Italian', 'Japanese','American', 'Indian', 
                'Mexican', 'Thai']
X['cuisines'] = X.loc[:,relevant_cuisines].sum(axis = 1)
AD = X[(X['cuisines'] > 0) & (X['review_count'] > 50)]
AD.shape

(935, 39)

### Look at statistics and write to file

In [65]:
AD.reset_index(drop=True).describe()

,stars,review_count,avg_checkins_weekday,avg_checkins_weekend,avg_checkins_Fri,avg_checkins_Mon,avg_checkins_Sat,avg_checkins_Sun,avg_checkins_Thu,avg_checkins_Tue,avg_checkins_Wed,Avg_checkins_Afternoon,Avg_checkins_Evening,Avg_checkins_Late_Night,Avg_checkins_Morning,Avg_checkins_Night,0_Afternoon,0_Evening,0_Late Night,0_Morning,0_Night,1_Afternoon,1_Evening,1_Late Night,1_Morning,1_Night,Nightlife,Bars,Canadian,Chinese,Italian,Japanese,American,Indian,Mexican,Thai,cuisines
count,935.000000,935.000000,935.000000,935.000000,935.000000,935.000000,935.000000,935.000000,935.000000,935.000000,935.000000,935.000000,935.000000,935.000000,935.000000,935.000000,935.000000,935.000000,935.000000,935.000000,935.000000,935.000000,935.000000,935.000000,935.000000,935.000000,935.000000,935.000000,935.000000,935.000000,935.000000,935.000000,935.000000,935.000000,935.000000,935.000000,935.000000
mean,3.581283,143.557219,2.884652,4.132965,3.577089,2.767905,4.460237,4.261640,3.102584,2.763442,2.897033,1.710614,3.060750,1.760086,0.827690,4.376812,1.246940,2.312587,1.255678,0.527502,3.700720,1.885876,3.782394,2.014439,0.705348,5.188978,0.300535,0.429947,0.195722,0.109091,0.139037,0.180749,0.171123,0.055615,0.070588,0.059893,1.712299
std,0.500345,132.322042,1.819993,2.924275,2.530763,1.900968,3.352242,3.316482,2.103295,1.829103,1.942901,1.699484,2.668727,1.296320,0.731739,3.842314,1.130459,1.982957,0.971621,0.771567,3.257492,2.670927,3.843912,1.920023,0.833923,4.757250,0.458736,0.495333,0.396968,0.311920,0.346171,0.385016,0.376818,0.229299,0.256273,0.237416,0.901823
min,1.500000,51.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,3.500000,69.500000,1.811012,2.321429,2.074176,1.607692,2.363636,2.250000,1.800000,1.666667,1.714286,1.000000,1.631579,1.000000,0.000000,2.246622,1.000000,1.285714,1.000000,0.000000,1.941176,1.000000,1.750000,1.000000,0.000000,2.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,3.500000,100.000000,2.363636,3.187500,2.875000,2.250000,3.384615,3.300000,2.500000,2.250000,2.333333,1.285714,2.272727,1.416667,1.000000,3.135135,1.000000,1.800000,1.000000,0.000000,2.714286,1.000000,2.625000,1.500000,1.000000,3.647059,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,4.000000,162.500000,3.290323,4.892720,4.230159,3.250000,5.402778,5.074176,3.559028,3.222222,3.384615,2.000000,3.469669,2.066667,1.000000,5.080703,1.500000,2.618056,1.600000,1.000000,4.170290,2.000000,4.279221,2.500000,1.000000,6.322917,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000
max,5.000000,1494.000000,20.367647,28.056604,29.705882,18.444444,29.631579,31.900000,21.470588,22.000000,19.888889,21.392857,27.714286,10.761905,9.800000,41.595238,12.937500,23.600000,8.250000,15.333333,34.375000,32.750000,42.400000,15.111111,6.250000,51.222222,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,5.000000


In [66]:
AD.to_csv('../data/clusteringAD.csv', index = False)